In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from textwrap import wrap
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.dates as mdates
from datetime import datetime
import ipaddress
import glob
from parallel_pandas import ParallelPandas
import seaborn as sns
import matplotlib.cm

In [2]:
pd.set_option('display.expand_frame_repr', False)

In [3]:
workdir = '/net/archive/telescope/dataframe'#/IMC-freeze'
workdir = './data'
#workdir = '/net/archive/telescope/dataframe'
#t1_f = 'telescope-t1.parquet'
date = '20240702'
t1_f = f'telescope-t1-{date}.parquet'
t1_file = f'{workdir}/{t1_f}'
t2_f = f'telescope-t2-{date}.parquet'
t2_file = f'{workdir}/{t2_f}'
t3_f = f'telescope-t3-{date}.parquet'
t3_file = f'{workdir}/{t3_f}'
t4_f = f'telescope-t4-{date}.parquet'
t4_file = f'{workdir}/{t4_f}'
#announcement_log_file = f'{workdir}/prefix-announcements.csv'
#figure_dir = '../figures/final'

In [53]:
#t1 = pd.read_parquet(t1_file)
t2 = pd.read_parquet(t2_file)
#t3 = pd.read_parquet(t3_file)
#t4 = pd.read_parquet(t4_file)

In [54]:
for t in [t1,t2,t3,t4]:
    print(t.Date.max())

2024-07-02 00:00:00
2024-07-03 00:00:00
2024-07-02 00:00:00
2024-07-02 00:00:00


In [6]:
addr_map01 = pd.read_csv('./addr_type_map',sep='|',index_col='ip_addr')
addr_map02 = pd.read_csv('./addr_type_map02',sep='|',names=['ip_addr','addr_type'],index_col='ip_addr')

In [7]:
addr_map = pd.concat([addr_map01,addr_map02])

In [8]:
addr_map.loc[addr_map.index.str.endswith('::'),'addr_type'] = 'full_zero_addr'

In [55]:
#for t in tqdm([t1,t2,t3,t4]):
for t in tqdm([t2]):
    t['dest_addr_type'] = t.Destination_Address.map(addr_map.addr_type)

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
t1.to_parquet(t1_file,compression=None)

In [11]:
t3.to_parquet(t3_file,compression=None)

In [12]:
t4_f2 = f'telescope-t4-{date}-withSpoki.parquet'
t4_file2 = f'{workdir}/{t4_f2}'
t4.to_parquet(t4_file2,compression=None)

In [13]:
t4_nospoki = t4[((t4.TCP_Flags=='nan') | (t4.TCP_Flags=='··········S·')) & (~t4.Source_Address.str.startswith('2a01:6427:'))]

In [14]:
t4_nospoki.to_parquet(t4_file,compression=None)

In [32]:
t1.dest_addr_type.value_counts(dropna=False)

dest_addr_type
randomized        31101725
low-byte           7582741
pattern-bytes      2105891
embedded-ipv4      1519763
full_zero_addr     1118665
ieee-derived         90843
embedded-port        89803
isatap                 217
Name: count, dtype: int64

In [33]:
t2[t2.Source_Address.str.startswith('2a05:')]

,Timestamp,Frame_Length,IP_Version,Flow_Label,Payload_Length,Next_Header,Hop_Limit,Source_Address,Destination_Address,Protocol,...,Session_ID_64,Interarrival_Time_48,Session_ID_48,is_oneoff_128,is_oneoff_64,is_oneoff_48,is_triggered_128,is_triggered_64,is_triggered_48,dest_addr_type


In [59]:
t2.period_128.value_counts(dropna=False)

period_128
-1.0        11224071
 11280.0     1486972
 1440.0      1300754
 720.0        302481
 9900.0       234660
              ...   
 13800.0          11
 6150.0           10
 43080.0           6
 25320.0           6
 4830.0            3
Name: count, Length: 447, dtype: int64

In [58]:
t2.period_128 = t2.period_128.fillna(-1)

In [60]:
t2_f2 = f'telescope-t2-{date}-withSpoki.parquet'
t2_file2 = f'{workdir}/{t2_f2}'
t2.to_parquet(t2_file2)

In [61]:
t2_nospoki = t2[((t2.TCP_Flags=='nan') | (t2.TCP_Flags=='··········S·')) & (~t2.Source_Address.str.startswith('2001:67c:254:'))]

In [70]:
object_columns = ['Source_Address', 'Protocol',
       'scan_source_128', 'scan_source_32',
       'scan_source_48', 'scan_source_64', 'Geo', 'Org','prefix_target',
        'scantool','DNS_id','Flow_Label','TCP_Flags']
object_columns = t2_nospoki.select_dtypes(include=['category']).columns
with tqdm(total=len(object_columns)) as pbar:
    for column in object_columns:
        t2_nospoki[column] = t2_nospoki[column].cat.remove_unused_categories()
        pbar.update(1)

  0%|          | 0/13 [00:00<?, ?it/s]

/tmp/ipykernel_6440/274015353.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2_nospoki[column] = t2_nospoki[column].cat.remove_unused_categories()


In [71]:
t2_f2 = f'telescope-t2-{date}-withDNS.parquet'
t2_file2 = f'{workdir}/{t2_f2}'
t2_nospoki.to_parquet(t2_file2)

In [63]:
t2_nodns = t2_nospoki[t2_nospoki.Destination_Address!='2001:67c:254::216']

In [72]:
object_columns = t2_nodns.select_dtypes(include=['category']).columns
with tqdm(total=len(object_columns)) as pbar:
    for column in object_columns:
        t2_nodns[column] = t2_nodns[column].cat.remove_unused_categories()
        pbar.update(1)

  0%|          | 0/13 [00:00<?, ?it/s]

/tmp/ipykernel_6440/2018571583.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2_nodns[column] = t2_nodns[column].cat.remove_unused_categories()


In [38]:
t2_nodns.Source_Address.nunique()

8201

In [40]:
dns_scanner = t2_nospoki[t2_nospoki.Destination_Address=='2001:67c:254::216'].Source_Address.unique()

In [55]:
t2_nodns[t2_nodns.Source_Address.isin(dns_scanner)].groupby('Source_Address',observed=True).agg(on_days=('Date','nunique'))

,on_days
Source_Address,
2001:550:9005:6000::11,7
2001:550:9005:e000::11,96
2408:4002:100d:8400:9c7a:7e5e:fdb2:43fa,35
240b:4001:112:7b00:79e3:688a:710c:6051,5
240b:4001:21b:1501:5b7a:abd3:5e30:3958,2


In [73]:
t2_nodns.to_parquet(t2_file)

In [51]:
tmp=t2_nospoki[t2_nospoki.Destination_Address=='2001:67c:254::216'].groupby('Source_Address').agg(on_days=('Date','nunique'))

In [53]:
tmp[tmp.on_days>1]

,on_days
Source_Address,
2001:1900:2200:b200::22,29
2001:19f0:4401:a7c:5400:2ff:fe2b:da78,4
2001:19f0:7001:3d61:5400:4ff:fedf:2ee7,7
2001:19f0:8000:1d31:5400:4ff:fed8:c3c2,2
2001:19f0:ac01:6cf:5400:3ff:fe97:51ee,41
...,...
2a13:a5c3:1000::,6
2a13:df80:7c10::d4,3
fe80::a86e:3cff:feae:c241,70


In [74]:
source_ips = pd.concat([t1.Source_Address.drop_duplicates(),t2_nodns.Source_Address.drop_duplicates(),t3.Source_Address.drop_duplicates(),t4_nospoki.Source_Address.drop_duplicates()],ignore_index=True).to_frame()

In [75]:
source_ips.drop_duplicates().to_csv('./sips4rdns-t1234.csv',index=False,header=False)

In [30]:
rdns = pd.read_csv('./sips4rdns-t134.csv.out',sep='|',names=['ip_addr','ptr_record'])

In [31]:
rdns

,ip_addr,ptr_record
0,2a05:e740:4163::2,No PTR record found
1,2a05:e740:9160::2,No PTR record found
2,2a05:e740:1946::2,No PTR record found
3,2a05:e740:1144::2,No PTR record found
4,2a05:e740:1947::2,No PTR record found
...,...,...
12084,2409:8950:621:c4db:4994:5659:f92e:1948,No PTR record found
12085,2607:9b00:2300:1100::1a61,No PTR record found
12086,2a02:50c0:601a:1711:2:78:40:133,No PTR record found
12087,2a0e:7b85:9a5a:8205:a3fe:3f2e:2e8f:2181,;; communications error to 8.8.8.8#53: timed out


In [52]:
for row in rdns[rdns.ptr_record!='No PTR record found'].iterrows():
    #if 'skynet' in str(row[1].ptr_record).lower():
    if not pd.isnull(row[1].ptr_record):
        print(f'{row[1].ip_addr} -- {row[1].ptr_record}')

2a10:3781:2851::1 -- 2a10-3781-2851.connected.by.freedominter.net.
2a02:1810:3d28:d000:da58:d7ff:fe03:be4 -- ptr-1u2a2xrgkka7a3kjwmc.18120a2.ip6.access.telenet.be.
2001:2000:6018::1412:818:4024 -- fi-hkihdc-asbr2.datanet.fi.
2a01:cb05:26c:e200:e65f:1ff:fe8e:5f6a -- 2a01cb05026ce200e65f01fffe8e5f6a.ipv6.abo.wanadoo.fr.
2001:4ca0:108:42:0:80:6:9 -- planetlab9.gino-research.net.in.tum.de.
2001:41d0:305:2100::902 -- vps-e1ebdf86.vps.ovh.net.
2603:8000:8e03:e601:ba27:ebff:fe4a:ca35 -- syn-2603-8000-8e03-e601-ba27-ebff-fe4a-ca35.res6.spectrum.com.
2003:dc:870b:7800:d66e:eff:fe23:5f34 -- p200300dc870b7800d66e0efffe235f34.dip0.t-ipconnect.de.
2a0a:a549:2ca2:1:fa1a:67ff:fe52:e518 -- 2a0a-a549-2ca2-1-fa1a-67ff-fe52-e518.ipv6dyn.netcologne.de.
2a0c:2f07:ac1::247 -- anycast-detection.b621.net.
2001:7f8:1e:3::184 -- ark-caida.sox.rs.
2a00:11c0:3:949::3 -- dns.nextdns.io.
2001:4ca0:108:42::1:9 -- planetlab9.gino-research.net.in.tum.de.
2001:56a:7920:100:ba27:ebff:feaf:2b24 -- node-1w7jr9yejg36r2mv7i